In [1]:
import transformers
from transformers import (
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    EvalPrediction,
    HfArgumentParser,
    PretrainedConfig,
    Trainer,
    TrainingArguments,
    default_data_collator,
    set_seed,
)
from transformers.trainer_utils import get_last_checkpoint, is_main_process
from transformers.utils import check_min_version

In [41]:
import json
import torch
import pickle
import random
import os
import json
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score,recall_score,f1_score,precision_score,classification_report,confusion_matrix


In [3]:
task_to_keys = {
    "cola": ("sentence", None),
    "mnli": ("premise", "hypothesis"),
    "mrpc": ("sentence1", "sentence2"),
    "qnli": ("question", "sentence"),
    "qqp": ("question1", "question2"),
    "rte": ("sentence1", "sentence2"),
    "sst2": ("sentence", None),
    "stsb": ("sentence1", "sentence2"),
    "wnli": ("sentence1", "sentence2"),
}

### Get Emoji Raw Data

In [15]:
raw_negative_data_list=[]
raw_positive_data_list=[]
raw_neutral_data_list=[]

for negative_data_path in os.listdir(f"./emoji_sentiment_data/raw_data/text_negative"):
    with open (f"../emoji_sentiment_data/raw_data/text_negative/{negative_data_path}", "r") as myfile:
        temp_list=myfile.read().splitlines()
        raw_negative_data_list.extend(temp_list)
    
for positive_data_path in os.listdir("./emoji_sentiment_data/raw_data/text_positive"):
    with open (f"../emoji_sentiment_data/raw_data/text_positive/{positive_data_path}", "r") as myfile:
        temp_list=myfile.read().splitlines()
        raw_positive_data_list.extend(temp_list)

for neutral_data_path in os.listdir("./emoji_sentiment_data/raw_data/text_neutral"):
    with open (f"../emoji_sentiment_data/raw_data/text_neutral/{neutral_data_path}", "r") as myfile:
        temp_list=myfile.read().splitlines()
        raw_neutral_data_list.extend(temp_list)


### Get Emoji Processed Data

In [12]:
processed_negative_data_list=[]
processed_positive_data_list=[]
processed_neutral_data_list=[]

with open (f"../emoji_sentiment_data/processed_data/negative/negative.txt", "r") as myfile:
    temp_list=myfile.read().splitlines()
    processed_negative_data_list.extend(temp_list)
    
with open (f"../emoji_sentiment_data/processed_data/positive/positive.txt", "r") as myfile:
    temp_list=myfile.read().splitlines()
    processed_positive_data_list.extend(temp_list)

with open (f"../emoji_sentiment_data/processed_data/neutral/neutral.txt", "r") as myfile:
    temp_list=myfile.read().splitlines()
    processed_neutral_data_list.extend(temp_list)
    

In [56]:
processed_negative_data_w_labels_list = [{"text":val,"label":"negative"} for val in processed_negative_data_list]
processed_positive_data_w_labels_list = [{"text":val,"label":"positive"} for val in processed_positive_data_list]
processed_neutral_data_w_labels_list = [{"text":val,"label":"neutral"} for val in processed_neutral_data_list]


In [57]:
X=processed_negative_data_w_labels_list + processed_positive_data_w_labels_list 
random.shuffle(X) 


In [58]:
X_train, X_test = train_test_split(X, test_size=0.2, random_state=10, shuffle= True)

X_train, X_val = train_test_split(X_train, test_size=0.25, random_state=5, shuffle=True) 

In [59]:
len(X_train)

100746

In [60]:
with open('/Users/aslihanuysal/Desktop/nlp_sentiment_analysis/emoji_sentiment_data/splitted_processed_data/2_classes/train.json', 'w') as outfile:
    json.dump(X_train, outfile)
    
with open('/Users/aslihanuysal/Desktop/nlp_sentiment_analysis/emoji_sentiment_data/splitted_processed_data/2_classes/test.json', 'w') as outfile:
    json.dump(X_test, outfile)  

with open('/Users/aslihanuysal/Desktop/nlp_sentiment_analysis/emoji_sentiment_data/splitted_processed_data/2_classes/validation.json', 'w') as outfile:
    json.dump(X_val, outfile)  
    
    

In [55]:
with open('/Users/aslihanuysal/Desktop/nlp_sentiment_analysis/emoji_sentiment_data/splitted_processed_data/3_classes/train.json', 'w') as outfile:
    json.dump(X_train, outfile)
    
with open('/Users/aslihanuysal/Desktop/nlp_sentiment_analysis/emoji_sentiment_data/splitted_processed_data/3_classes/test.json', 'w') as outfile:
    json.dump(X_test, outfile)  

with open('/Users/aslihanuysal/Desktop/nlp_sentiment_analysis/emoji_sentiment_data/splitted_processed_data/3_classes/validation.json', 'w') as outfile:
    json.dump(X_val, outfile)  
    
    

In [43]:
model_name_or_path="savasy/bert-base-turkish-sentiment-cased"
do_train=True
#do_eval=True
#do_predict=True
per_device_train_batch_size=4
per_device_eval_batch_size=4
gradient_accumulation_steps=4
num_train_epochs=4
max_seq_length=512
#load_best_model_at_end=
#evaluation_strategy epoch=
#save_strategy epoch=
metric_for_best_model="loss"
overwrite_output_dir=True
seed=1
cache_dir='./.cache' 
logging_dir="./bert-base-turkish-sentiment-cased" 
output_dir="./bert-base-turkish-sentiment-cased"
pad_to_max_length=True
last_checkpoint=None
resume_from_checkpoint=None
max_train_samples=None

In [ ]:
unique_label_list=["positive","negative","neutral"]

In [41]:
train_dataset=X_train
eval_dataset=X_val

In [ ]:
#true_labels = datasets["test"]["label"]

In [44]:
config = AutoConfig.from_pretrained(
    model_name_or_path,
    num_labels=2,
    #finetuning_task=task_name,
    cache_dir=cache_dir
)
tokenizer = AutoTokenizer.from_pretrained(
    "savasy/bert-base-turkish-sentiment-cased",
    cache_dir=cache_dir
    #use_fast=model_args.use_fast_tokenizer
)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name_or_path,
    #from_tf=bool(".ckpt" in model_args.model_name_or_path),
    config=config,
    cache_dir=cache_dir
)

In [ ]:
# Initialize our Trainer
trainer = MultilabelTrainer(
    model=model,
    args=training_args,
    train_dataset=,
    eval_dataset=,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    #data_collator=data_collator,
)



# Training
if do_train:
    checkpoint = None
    if resume_from_checkpoint is not None:
        checkpoint = resume_from_checkpoint
    elif last_checkpoint is not None:
        checkpoint = last_checkpoint
        
    train_result = trainer.train()
    metrics = train_result.metrics
    #max_train_samples = (
    #    max_train_samples if max_train_samples is not None else len(train_dataset)
    #)
    
    metrics["train_samples"] = min(max_train_samples, len(train_dataset))

    trainer.save_model()  # Saves the tokenizer too for easy upload

    trainer.log_metrics("train", metrics)
    trainer.save_metrics("train", metrics)
    trainer.save_state()

In [ ]:
if data_args.pad_to_max_length:
        padding = "max_length"